In [1]:

# Azure Blob Storage credentials
AZURE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=usereadydemo;AccountKey=gI4MM9CckLtAIqcOAOfZlBVV3ZMSb0Lj/GgkEI+3cW4J8IRBnFYeDJtToJe9SVor/COOF6xPIG2j+AStLy/0ww==;EndpointSuffix=core.windows.net"
CONTAINER_NAME = "data"

# Connect to Azure Blob Storage



In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from azure.storage.blob import BlobServiceClient
import shutil

def load_file(file_name):
    loader=[]
    print(file_name.split(".")[-1])
    if file_name.split('.')[-1] == "pptx":
        loader = UnstructuredPowerPointLoader(file_name).load()
    elif file_name.split('.')[-1] == "pdf":
        loader = PyPDFLoader(file_name).load()    
    elif file_name.split('.')[-1] == "docx":
        loader = Docx2txtLoader(file_name).load()
    elif file_name.split('.')[-1] == "html":
        loader = UnstructuredHTMLLoader(file_name).load()
    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=0,
            is_separator_regex=False,
        )
    pages = text_splitter.split_documents(loader)
    return pages

import os
def file_to_chunks():
    pages=[]
    for file_name in os.listdir("Azure_data"):
        m=load_file(f"Azure_data\\{file_name}")
        print(f"Azure_data\\{file_name} is being processed to documents.....")
        print(len(m))
        pages.extend(m)
    # shutil.rmtree("Azure_data")
    return pages
    
def azure_data_download(AZURE_CONNECTION_STRING,CONTAINER_NAME):
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(CONTAINER_NAME)
    if not os.path.exists("Azure"):
        os.mkdir("Azure_data")
    for file_name in container_client.list_blobs():
        blob_client = container_client.get_blob_client(file_name)
        with open(f"Azure_data\\{file_name.name}", "wb") as file:
            data = blob_client.download_blob().readall()
            file.write(data)

In [5]:
azure_data_download(AZURE_CONNECTION_STRING,CONTAINER_NAME)

In [6]:
files = file_to_chunks()

pptx
Azure_data\Free_Test_Data_1MB_PPTX.pptx is being processed to documents.....
27
pdf
Azure_data\Natu Lauchande - Machine Learning Engineering with MLflow_ Manage the end-to-end machine learning life cycle with MLflow (2021, Packt Publishing) - libgen.li.pdf is being processed to documents.....
1171
docx
Azure_data\sample1.docx is being processed to documents.....
43
docx
Azure_data\sample2.docx is being processed to documents.....
3
html
Azure_data\sample2.html is being processed to documents.....
2
docx
Azure_data\sample4.docx is being processed to documents.....
2334


In [7]:
len(files)

3580

In [8]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv  import load_dotenv
load_dotenv()
db = Chroma.from_documents(files, OpenAIEmbeddings(), persist_directory="Chroma_db",collection_metadata={"name":"data"})

In [10]:
db